# Detección de emociones multimodal

## Text

In [24]:
import joblib

with open('./transcriptions/output.txt', "r") as archivo:
    contenido = archivo.read()

texto = contenido

loaded_model = joblib.load('textModel.pkl')
prediccionTexto = loaded_model.predict([texto])[0]
print("Emotion detection from Text: " + prediccionTexto)

Emotion detection from Text: neutral


## Video

In [22]:
import cv2
import numpy as np
from keras.models import model_from_json


emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
emotion_times = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}

# load json and create model
json_file = open('emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

# load weights into new model
emotion_model.load_weights("emotion_model.h5")
print("Loaded model from disk")

# start the webcam feed
#cap = cv2.VideoCapture(0)

# pass here your video path
# you may download one from here : https://www.pexels.com/video/three-girls-laughing-5273028/
cap = cv2.VideoCapture("/home/juanespadi/Tesis/multimodal_ws/src/recording_tools/resources/videos/video.avi")

while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    try:
        frame = cv2.resize(frame, (1280, 720))
    except:
        break
    
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)

        maxindex = int(np.argmax(emotion_prediction))
        if maxindex != 4:
            emotion_times[maxindex] += 1

        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


totalValues = list(emotion_times.values())
print("Emotion detection from Video: " + emotion_dict[np.argmax(totalValues)])

cap.release()
cv2.destroyAllWindows()

Loaded model from disk
1/1 [==============================] - 0s 20ms/step
Emotion detection from Video: Fearful


## Audio

In [21]:
import soundfile
import librosa
import glob
import numpy as np
import joblib

#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

archivo = glob.glob("/home/juanespadi/Tesis/multimodal_ws/src/recording_tools/resources/audios/audio.wav")[0]

print(archivo)

featurexd = extract_feature(archivo, mfcc=True, chroma=True, mel=True)


modeloAudio = joblib.load('audioModel.pkl')

featurexd = featurexd.reshape(1, -1)
prediccion = modeloAudio.predict(featurexd)
print("Emotion detection from Audio: " + prediccion[0])

/home/juanespadi/Tesis/multimodal_ws/src/recording_tools/resources/audios/audio.wav
Emotion detection from Audio: angry
